In [16]:
!pip install pandas tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd

In [33]:
df = pd.read_csv('_УУ Себестоимость Май2025 (для дэшборда) - Май 2025.csv', skiprows=1)

In [34]:
df

,Период,Артикул,Описание,Категория,"Себестоимость на апрель\n (С НДС, рубли)",Себестоимость единицы\n 4_25 CNY с НДС,Unnamed: 6
0,март 2025,1_Кастрюли,PT1-1,Кастрюли 3 шт,1382,106.41,NaN
1,март 2025,1_Кастрюли,PT1-2,Кастрюли 3 шт + сковорода,2287,176.12,NaN
2,март 2025,1_Кастрюли,PT1-3,Кастрюли 4 шт,2377,183.00,NaN
3,март 2025,1_Кастрюли,PT1-4,Набор 2 кастрюли + ковшик,1662,128.00,NaN
4,март 2025,1_Кастрюли,PT1-5,Набор кастрюль 3 шт + ковшик,2429,187.00,NaN
...,...,...,...,...,...,...,...
78,март 2025,Прочее,PHA1-1GR,ФОТОАЛЬБОМ С НАКЛЕЙКАМИ (А4).,661,NaN,---
79,март 2025,Прочее,PHA1-1PK,ФОТОАЛЬБОМ С НАКЛЕЙКАМИ (А4).,661,NaN,---
80,март 2025,Прочее,PHA1-1RD,ФОТОАЛЬБОМ С НАКЛЕЙКАМИ (А4).,661,NaN,---
81,март 2025,Прочее,SHC1-1BK,СТЕЛЛАЖ В ПРИХОЖУЮ ДЛЯ ХРАНЕНИЯ ОБУВИ,704,NaN,---


In [24]:
dbname='app_db'
user='user'
host='93.183.73.107'
port=57212

In [7]:
import getpass
password = getpass.getpass()

 ········


In [35]:
from psycopg2 import connect, sql
from functools import wraps
from typing import Callable
from datetime import datetime, timedelta
from tqdm import tqdm

def inject_closable_cursor(func: Callable):
    @wraps(func)
    def wrapper(*args, **kwargs):
        connection = connect(
            dbname=dbname,
            user=user,
            password=password,
            host=host,
            port=port
        )
        try:
            with connection.cursor() as cursor:
                result = func(*args, cursor=cursor, **kwargs)
                connection.commit()
                return result
        except Exception:
            connection.rollback()
            raise
        finally:
            connection.close()
    return wrapper


@inject_closable_cursor
def save_cost_data(data, **kwargs):
    cursor = kwargs.get('cursor')
    for _, record in data.iterrows():
        cursor.execute(
            sql.SQL('''
                INSERT INTO cost (sku, date, cost_in_kopecks)
                VALUES (%s, %s, %s)
                ON CONFLICT (sku, date)
                DO UPDATE
                SET cost_in_kopecks = EXCLUDED.cost_in_kopecks
                WHERE cost.cost_in_kopecks IS NULL OR EXCLUDED.cost_in_kopecks != 0
            '''),
            [record['sku'], record['date'], record['cost_in_kopecks']]
        )

@inject_closable_cursor
def save_cost_data(data, **kwargs):
    cursor = kwargs.get('cursor')
    
    # Преобразуем "Себестоимость на апрель\n (С НДС, рубли)" в копейки и переименуем столбцы
    data = data.rename(columns={
        'Описание': 'sku',
        'Себестоимость на апрель\n (С НДС, рубли)': 'cost'
    })
    data['cost_in_kopecks'] = (data['cost'] * 100).astype(int)
    
    start_date = datetime(2025, 4, 1)
    end_date = datetime(2025, 4, 30)
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Для каждой даты добавляем данные из DataFrame
    for date in tqdm(date_range):
        for _, record in data.iterrows():
            cursor.execute(
                sql.SQL('''
                    INSERT INTO cost (sku, date, cost_in_kopecks)
                    VALUES (%s, %s, %s)
                    ON CONFLICT (sku, date)
                    DO UPDATE
                    SET cost_in_kopecks = EXCLUDED.cost_in_kopecks
                    WHERE cost.cost_in_kopecks IS NULL OR EXCLUDED.cost_in_kopecks != 0
                '''),
                [record['sku'], date.date(), record['cost_in_kopecks']]
            )

In [36]:
save_cost_data(df)

100%|██████████| 30/30 [03:15<00:00,  6.52s/it]
